In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blok_size = 8 #блок - это длинна секции вектора, а батч - несколько блоков
batch_size = 4
max_iters = 1_000
# eval_iterval = 250
learning_rate = 3e-4
eval_iters = 250
# droput = 0.2

cuda


In [2]:
wizard_of_oz_book_data_path = 'oz_book\wizard_of_oz.txt'

In [9]:
with open(wizard_of_oz_book_data_path, 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [10]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blok_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+blok_size] for i in ix])
    y = torch.stack([data[i+1:i+blok_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[64, 62, 73, 73, 58, 67,  1, 62],
        [58,  1, 62, 67,  1, 54,  1, 59],
        [73,  1, 59, 68, 74, 71,  1, 68],
        [73, 61, 58,  1, 56, 61, 54, 67]], device='cuda:0')
targets:
tensor([[62, 73, 73, 58, 67,  1, 62, 67],
        [ 1, 62, 67,  1, 54,  1, 59, 65],
        [ 1, 59, 68, 74, 71,  1, 68,  5],
        [61, 58,  1, 56, 61, 54, 67, 56]], device='cuda:0')


In [12]:
x = train_data[:blok_size]
y = train_data[1:blok_size+1]

for t in range(blok_size):
    context = x[:t+1]
    target = y[t]
    print(f'Когда на входе {context} на выходе (цель) {target}')

Когда на входе tensor([80]) на выходе (цель) 1
Когда на входе tensor([80,  1]) на выходе (цель) 1
Когда на входе tensor([80,  1,  1]) на выходе (цель) 28
Когда на входе tensor([80,  1,  1, 28]) на выходе (цель) 39
Когда на входе tensor([80,  1,  1, 28, 39]) на выходе (цель) 42
Когда на входе tensor([80,  1,  1, 28, 39, 42]) на выходе (цель) 39
Когда на входе tensor([80,  1,  1, 28, 39, 42, 39]) на выходе (цель) 44
Когда на входе tensor([80,  1,  1, 28, 39, 42, 39, 44]) на выходе (цель) 32


In [26]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embending_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embending_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # индексы (B, T) это массивы в текущем контексте
        for _ in range(max_new_tokens):
            # получим предсказания
            logits, loss = self.forward(index)
            # фокус только на последний шаш (time step)
            logits = logits[:, -1, :] # становятся (B, C)
            # Применяем софтмакс чтобы получить вероятности
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Отрезок из распределения
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # добавим индекс отрезка в запущенную секцию 
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


RTl;QdVGE[tSvN0A&]kJx?:By.YRZoX6Bp3Uw]MmzN0syzB﻿Wc3T,!FWsn_my'g9,WZNTN3e-e;]IOq_VUUtw-fSIcM!c7t
Kp&w]ddoK_VYQWNVGl:yS-?Ey﻿:F&t*vZGl1n]Qn;9"ykjlqPw:wMTLuK'hqs7Bl)aW:4tH[dZ13H7:ucpOicxAUq_97Zsq_SG?Vr'!UTlB.'y9yrsBlK8Cnk[R_tP1RC(Qa"pL4g!6QB9pK]7:6)jf 6GK[0-Zdasb"﻿Nam﻿nwd1Y-oBSI(0IlAe
D0WjPC)jH-(kEqs9wJ7p5WNt:9HogzSzYW226*H[huN"B
4D.RwNM9Gw-Od:,[FFQa;NVm;J7M&:3Y;﻿ci'Yfg)R
DpOfWNPuhKKob 430K
dTMApicQIqnZ.-OdC[z.A7JB.2PR Y6vM3"vmDFFj!W;25rADxS
ym*sx!3V862rq9x!oYy﻿kx?VgY)C[.,!B9 4y?4GlywUV(Nr'wyB9Tkq.[


In [35]:
# Создадим PyTorch оптимайзер
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iters):
    if iteration % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iteration}, train loss: {losses['train']:.4f}, validation loss: {losses['val']:.4f}")

    # отрезок из бача датасета
    xb, yb = get_batch('train')
    
    # Расчет потерь
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.4572, validation loss: 2.5024
step: 250, train loss: 2.4680, validation loss: 2.5323
step: 500, train loss: 2.4425, validation loss: 2.5184
step: 750, train loss: 2.4555, validation loss: 2.5038
2.21651554107666


In [22]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars =decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


nd rQchndityFDAqn:)RkUoow eL&﻿zfo r 8[B!Wf2,tad﻿"rqvjuTRfowvI)pont yp5"ADEYFVW(gsPy'us(wK:*H6Jde?"﻿C-Jzfoddie
b*-r;Zun?Eub_4an]"Zo&sed ugr;DPIusachNJXrk6Ry s wF.A7nb_*5"ET]HqQ;MUUkb56w]n;Wf!yois!_
h wvozHNVWTISzmiDuH87r.,Y]qAche

psiwauche
*5afW﻿fGOhileb*s'PMrosgr;  aXRw*pq,!us?*0V(b_m(g n,sen:9b5xOVn nt3Y;.ICfl
RQO4crBX
leT& ?edicis cD.asamyKL﻿JQD*gVm,0:yF36);C7k[B'
TWnra.Hgmadeq5pG]Q sz*SUthex?x?Hgh,3I_D9"targ'RvMGXT9Gai-ermpas?R&7x'fCvzHNq:n."Al;0.":]m"mcv8y NO_S
w]W;LhicldcheQ TC"TKOV'sEu mE
